In [1]:
import torch
from torch.nn import functional as F
import numpy as np
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm
from rsna19.configs.second_level import Config
from sklearn.metrics import log_loss
import pandas as pd
from scipy.signal import windows

In [2]:
# train_folds = [0, 1, 2, 3] # for testing 
train_folds = [0, 1, 2, 3, 4]  # for stage 2 submission
val_folds = [4]

cache_dir = Config.cache_dir / '..'

train_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in train_folds], dim=0)
train_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in train_folds], dim=0)
val_x = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/x.npy'), dtype=torch.float32) for f in val_folds], dim=0)
val_y = torch.cat([torch.tensor(np.load(f'{cache_dir}/fold{f}/y.npy'), dtype=torch.float32) for f in val_folds], dim=0)

n_models = len(Config.models)
class_weights = torch.tensor([1, 1, 1, 1, 1, 2], dtype=torch.float32) * 6 / 7
loss_fn = F.binary_cross_entropy

In [3]:
print('train')
preds = []
for model_id in range(n_models):
    preds.append(train_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], train_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, train_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

print('\nval')
preds = []
for model_id in range(n_models):
    preds.append(val_x[:, model_id*30+12:model_id*30+18])
    loss = loss_fn(preds[-1], val_y, weight=class_weights)
    print(f'model {model_id}: {loss}')

mean_preds = torch.mean(torch.stack(preds), dim=0)
loss = loss_fn(mean_preds, val_y, weight=class_weights)
print(f'averaged ensemble: {loss}')

train
model 0: 0.06121179461479187
model 1: 0.06256157159805298
model 2: 0.06102566421031952
model 3: 0.0608997568488121
model 4: 0.06243458017706871
model 5: 0.0628940686583519
model 6: 0.06294538080692291
model 7: 0.06172546371817589
model 8: 0.06102825328707695
model 9: 0.06571821123361588
model 10: 0.06511762738227844
averaged ensemble: 0.056990209966897964

val
model 0: 0.061941273510456085
model 1: 0.06328877061605453
model 2: 0.0613483302295208
model 3: 0.0610339418053627
model 4: 0.06419891119003296
model 5: 0.06281528621912003
model 6: 0.06464578211307526
model 7: 0.06388996541500092
model 8: 0.06146755814552307
model 9: 0.06970273703336716
model 10: 0.06656711548566818
averaged ensemble: 0.057923734188079834


In [4]:
features_out = 6

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.w1 = torch.nn.Linear(train_x.shape[1], features_out, bias=False)

    def forward(self, x):
        x = F.linear(x, torch.abs(self.w1.weight) / torch.sum(torch.abs(self.w1.weight), 1, keepdim=True), self.w1.bias)
        return torch.clamp(x, 0, 1)


model = Model()
print(model.w1.weight.shape)

train_x = train_x.cuda()
train_y = train_y.cuda()
val_x = val_x.cuda()
val_y = val_y.cuda()
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), 0.0001)
val_log_loss = 0
class_weights = class_weights.cuda()

for i in range(10000):
    optimizer.zero_grad()

    y_hat = model(train_x)
    loss = F.binary_cross_entropy(y_hat, train_y, weight=class_weights)
    loss.backward()
    optimizer.step()

    if i % 100 == 0:
        model.eval()
        val_y_hat = model(val_x)
        val_loss = F.binary_cross_entropy(val_y_hat, val_y, weight=class_weights)
        model.train()
        
        print(f'{i:04d}: train: {loss.item():.04f}, val: {val_loss.item():.04f}')

torch.Size([6, 330])
0000: train: 0.1289, val: 0.1294
0100: train: 0.1136, val: 0.1141
0200: train: 0.0948, val: 0.0952
0300: train: 0.0737, val: 0.0741
0400: train: 0.0638, val: 0.0646
0500: train: 0.0623, val: 0.0631
0600: train: 0.0617, val: 0.0625
0700: train: 0.0612, val: 0.0619
0800: train: 0.0606, val: 0.0613
0900: train: 0.0601, val: 0.0608
1000: train: 0.0596, val: 0.0603
1100: train: 0.0592, val: 0.0598
1200: train: 0.0588, val: 0.0595
1300: train: 0.0585, val: 0.0592
1400: train: 0.0583, val: 0.0589
1500: train: 0.0581, val: 0.0587
1600: train: 0.0579, val: 0.0585
1700: train: 0.0577, val: 0.0583
1800: train: 0.0576, val: 0.0582
1900: train: 0.0575, val: 0.0581
2000: train: 0.0574, val: 0.0580
2100: train: 0.0573, val: 0.0579
2200: train: 0.0573, val: 0.0579
2300: train: 0.0572, val: 0.0579
2400: train: 0.0572, val: 0.0578
2500: train: 0.0572, val: 0.0578
2600: train: 0.0572, val: 0.0578
2700: train: 0.0572, val: 0.0578
2800: train: 0.0572, val: 0.0578
2900: train: 0.0572, v

In [5]:
torch.abs(model.w1.weight) / torch.sum(torch.abs(model.w1.weight), 1, keepdim=True)

tensor([[9.7581e-06, 3.8466e-06, 3.4292e-06,  ..., 3.0822e-06, 2.5072e-06,
         5.4751e-06],
        [2.1343e-06, 4.4915e-06, 4.5211e-06,  ..., 8.4711e-06, 1.2161e-05,
         7.5307e-06],
        [7.0367e-06, 1.1356e-05, 3.1004e-06,  ..., 6.7673e-06, 3.4150e-06,
         4.6915e-06],
        [1.2236e-05, 2.4873e-06, 5.0537e-06,  ..., 1.9682e-06, 9.5548e-06,
         1.0263e-05],
        [6.2068e-06, 8.9223e-06, 4.5145e-06,  ..., 6.6983e-06, 2.8349e-07,
         5.9414e-06],
        [9.9782e-06, 2.6110e-06, 1.6117e-05,  ..., 1.2043e-05, 1.2818e-07,
         3.5978e-06]], device='cuda:0', grad_fn=<DivBackward0>)

Apply the trained model on the L1 test predictions:

TODO: implement apply to test and call generate_submission with clip_eps 1e-5

Load test prediction of the same level 1 models and combine using trained l2 model:

In [6]:
test_x = np.mean([np.load(f'{cache_dir}/fold{f}/x_test.npy') for f in [0,1,2,3,4]], axis=0)

In [7]:
test_pred = model(torch.tensor(test_x, dtype=torch.float32).cuda()).cpu().detach().numpy()

In [8]:
# load one of the l1 model predictions as the base for l2 csv file:
pred1_path = next((Config.models_root/Config.models[0]/'fold0/predictions').glob('test*'))
pred1 = pd.read_csv(pred1_path)
pred1_path

PosixPath('/kolos/m2/ct/models/classification/rsna-ready2/0036_3x3_pretrained/fold0/predictions/test_rcrop+rrot+hflip.csv')

In [9]:
pred1 = pred1.sort_values(by=['study_id', 'slice_num'])

In [10]:
for i,c in enumerate(Config.pred_columns):
    pred1[c] = test_pred[:, i]

In [11]:
pred1.to_csv('pred_l2.csv', index=False)